In [7]:
!pip3 install boto3 --user
!pip3 install kafka-python --user

  Using cached https://files.pythonhosted.org/packages/82/39/aebe3ad518513bbb2260dd84ac21e5c30af860cc4c95b32acbd64b9d9d0d/kafka_python-1.4.6-py2.py3-none-any.whl


In [11]:
import json
import boto3
import time

from kafka import KafkaProducer
from kafka.errors import KafkaError
from kafka import KafkaConsumer

dynamodb = boto3.resource('dynamodb')
table_name = 'News-Feed'

In [4]:
def read_all_news():
    table = dynamodb.Table(table_name)
    response = table.scan()['Items']
    response.sort(key=lambda r : r['NewsID'], reverse=True)
    return json.dumps(response)

In [5]:
read_all_news()

'[{"NewsID": "1555817398.4438746", "FilePath": "test/1234.txt"}, {"NewsID": "1555732764.9522707", "FilePath": "test/1234.txt"}]'

In [17]:
def write_to_dynamo(data):
    table = dynamodb.Table(table_name)
    dynamo_data_actual = { key : value for key, value in data.items() if value }
    response = table.put_item(Item=dynamo_data_actual)
    return json.dumps({ 'title' : data['title'] })

In [21]:
consumer = KafkaConsumer(
    'testNews',
     bootstrap_servers=['3.18.83.70:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=False,
     group_id=str(time.time())
)

In [22]:
for message in consumer:
    key, value = message.key, message.value.decode('utf-8')
    try:
        write_to_dynamo(json.loads(value))
    except Exception as e:
        print(e)

KeyboardInterrupt: 

In [20]:
consumer.close()